# Stock Price Predictions

In [ ]:
# import modules
from datetime import datetime
!pip install yfinance --quiet
import yfinance as yf
import matplotlib.pyplot as plt
import datetime
import time
from pandas_datareader import data as pdr
import yfinance as yf
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, losses, callbacks
from tensorflow.keras.preprocessing.sequence import  TimeseriesGenerator
import pandas as pd
import numpy as np
import os 

In [ ]:
main_path = os.getcwd()
print(main_path)

In [ ]:
def get_stock_symbols():
    '''
    get stock symbols from NASDAQ and Indian Stock Market

    '''
    
    try:
        # getting the stock symbols of Indian Companies
        indian_stock_symbols = pd.read_html('https://indiancompanies.in/listed-companies-in-nse-with-symbol/')[0]

        #making first row as header /column names
        indian_stock_symbols.columns = indian_stock_symbols.iloc[0]
        indian_stock_symbols = indian_stock_symbols[1:]

        #Saving file
        indian_stock_symbols.to_csv('indian_stock_symbols.csv')

        # Nasdaq requires manual downloaing
        ## nasdaq_url = 'https://www.nasdaq.com/market-activity/stocks/screener'

        # Moving csv to src folder
        if  os.path.exists(os.path.join(main_path,'src')):
            pass
        else:
            os.makedirs('src')

            
        os.system('mv -f indian_stock_symbols.csv src/indian_stock_symbols.csv ')

    except Exception as e:
        print(e,' shit!!!')

In [ ]:
#updating the list
get_stock_symbols()

In [ ]:
# os.system('rm -rf src')

In [ ]:
# creating data path and Moving downloaded files
if not os.path.exists(os.path.join(main_path,'data')):
    os.makedirs('data')

comp1 = 'AAPL'

Hist = 5


In [ ]:
companies = [comp1]

In [ ]:
companies_data = {}
for comp in companies:
    yf.pdr_override() # <== that's all it takes :-)

    # download dataframe
    years = 5
    end_date = str(datetime.date.today()) # today
    start_date = str(datetime.date.today() - datetime.timedelta(days= years*365)) # some years from now
    
    data = pdr.get_data_yahoo(comp, start=start_date, end=end_date)
    companies_data[comp]=data

[*********************100%***********************]  1 of 1 completed


In [ ]:
df = companies_data['AAPL']

In [ ]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-09-19,149.309998,154.559998,149.100006,154.479996,154.479996,81474200
2022-09-20,153.399994,158.080002,153.080002,156.899994,156.899994,107689800
2022-09-21,157.339996,158.740005,153.600006,153.720001,153.720001,101696800
2022-09-22,152.380005,154.470001,150.910004,152.740005,152.740005,86652500
2022-09-23,151.190002,151.470001,148.559998,150.429993,150.429993,95939200


## Creating dataset

We need to create a dataset that moves with time and creates windows(features) and Horizons(label)

1. One to one dataset (One feature column and One Label)
    
    e.g. feature Adj Close --> Adj Close
2. Many to One dataset (All the Features and One Label)
    
    e.g. [Open, High, Low, Close, Adj Close, Volume] --> Adj Close

In [ ]:
# setting a columns to be predicted
wanted = 'Adj Close'
price = df[wanted].to_numpy()
time = df[wanted].index.to_numpy()

# show some prices
print(price)

[ 36.59358597  36.36819077  36.56748581 ... 153.72000122 152.74000549
 150.42999268]


In [ ]:
len(price)

1257

In [ ]:
WINDOW_SIZE = 3
HORIZON = 1
class WindowMaker:
    '''
    dataclass returns windows and horizons with given parameter

    -------------------
    Parameters:
        data:nd array       :   numpy array or pandas dataframe
        window_size:int     :   size of the window
        horizon_size:int    :   size of the horizon

        stride:int          :   how much window slides on data
        shift:int           :   gap between window and horizon(if needed) 

    '''
    from sklearn.model_selection import train_test_split

    def __init__(self, data , window_size:int=7, horizon_size:int=1, label_columns_indices:list=None,label_columns_names:list = None, stride:int = 1, shift:int=0):
        self.data = data
        self.window_size = window_size
        self.horizon_size = horizon_size
        self.label_columns_indices = label_columns_indices
        self.label_columns_names = label_columns_names
        self.stride = stride
        self.shift = shift





        if type(self.data) == pd.core.frame.DataFrame:

            self.column_indices = {name: i for i, name in
                                enumerate(self.data.columns)}

            if self.label_columns_names is not None:
                self.label_columns_indices = [self.column_indices[name]  for name in
                                        self.label_columns_names]
            
            # converting dataframe to ndarray
            self.data = self.data.to_numpy()


        # create a array of window size
        self.window_size_array = np.arange(self.window_size + self.shift + self.horizon_size)

        # create array with a forward shift
        self.all_array_index = np.arange(len(self.data)- (self.window_size + self.shift + self.horizon_size -1), step = self.stride).reshape(-1,1) + self.window_size_array

        # Spliting index in window and horizon
        self.all_windows_index = self.all_array_index[:, :self.window_size]
        self.all_horizons_index = self.all_array_index[:, self.window_size+self.shift:]


        # actuall values of horizon and windows
        self.all_windows = self.data[self.all_windows_index]
        self.all_horizons_full = self.data[self.all_horizons_index] # contains all the labels

        # Work out the label column indices.
        if self.label_columns_indices is not None:
            self.all_horizons = self.all_horizons_full[...,self.label_columns_indices]
        
        else:
            self.all_horizons = self.all_horizons_full


    def make_split(self):
        return self.all_windows, self.all_horizons

    # returns train, test or train, val, test as tuple
    def train_test_val(self,test_size:float = .1, val_size:float = None,random = False, random_state:int = 2):
        if random == True:
            # creating test data of test_size
            if test_size is not None:
                xtrain_rest, xtest, ytrain_rest, ytest = train_test_split(self.all_windows, self.all_horizons, test_size = test_size, random_state = random_state)
                if val_size is None:
                    return (xtrain_rest, ytrain_rest), (xtest, ytest)
                if val_size is not None:
                    xtrain, xval, ytrain, yval = train_test_split(xtrain_rest, ytrain_rest, test_size = val_size, random_state = random_state)
                    return (xtrain, ytrain),( xval, yval), (xtest, ytest)
            else:
                print('To just create val data use test_size')

        else:
            # creating test data of test_size
            if test_size is not None:
                test_len = int(len(self.all_windows)* test_size)
                xtrain_rest, xtest, ytrain_rest, ytest = self.all_windows[:-test_len], self.all_windows[-test_len:], self.all_horizons[:-test_len], self.all_horizons[-test_len:]
                if val_size is None:
                    return (xtrain_rest, ytrain_rest), (xtest, ytest)
                if val_size is not None:

                    val_len = int(len(xtrain_rest)* val_size)
                    xtrain, xval, ytrain, yval = xtrain_rest[:-val_len], xtrain_rest[-val_len:], ytrain_rest[:-val_len], ytrain_rest[-val_len:]
                    return (xtrain, ytrain),( xval, yval), (xtest, ytest)
            else:
                print('To just create val data use test_size')            

        

    def plot(self, plot_cols:list=None, model = None, figsize = (15,5),**kwargs):
        # plots data
        plt.figure(figsize = figsize)
        plt.grid(True)
        for i, name in enumerate(plot_cols):
            plt.subplot(len(plot_cols),1, i+1)
            plt.plot(self.all_horizons_full[...,self.column_indices[name]], **kwargs)
            plt.title(name)
            # if model is not None:
            #     predictions = model.pred
            # plt.plot(self.all_horizons_full[...,self.columns_indices[name]])
        plt.show()

    def __repr__(self):
        return '\n'.join([
                f'Total window size: {self.window_size}',
                f'Horizon size: {self.horizon_size}',            # if model is not None:
            #     predictions = model.pred
            # plt.plot(self.all_horizons_full[...,self.columns_indices[name]])
                f'Stride: {self.stride}',
                f'Shift: {self.shift}'])

In [ ]:
def compile_and_fit(model, train_data, val_data = None, epochs = 100, patience = 5, verbose = 1, ckpt_verbose = 1):

    # callbacks
    EarlyStopping = callbacks.EarlyStopping(monitor='mae', patience = patience )

    filepath = os.path.join('modelckpt', model.name)

    try:
        os.makedirs(filepath)
    except:
        pass

    ModelCheckpoint = callbacks.ModelCheckpoint(filepath= filepath,
                                                monitor = 'val_loss',
                                                verbose = ckpt_verbose,
                                                save_best_only = True,
                                                save_weights_only = True)
    
    # compile
    model.compile(loss = 'mae',
                  optimizer = 'adam',
                  metrics = ['mse', 'mae'])
    
    history = model.fit(train_data, epochs = epochs, 
                        validation_data= val_data,
                        batch_size = 32,
                        verbose = verbose,
                        callbacks = [ModelCheckpoint])
    print('metrics order :Loss, MSE, MAE')
    print('Train data Evaluation: ', model.evaluate(train_data))
    print('Validation data Evaluation: ',model.evaluate(val_data))
    
    return history

In [ ]:
def pred_plot(model, val_data, figsize = (10,7)):
    x= val_data.unbatch().as_numpy_iterator()
    yval = [i[1] for i in list(x)]
    xval = [i[0] for i in list(x)]

    # bigpred = []
    print(xval)
    ypred = tf.squeeze(model.predict(val_data))

    # print(np.arange(1,len(ypred)+1))
    #plot it 
    plt.figure(figsize = figsize)
    plt.grid(True)
    plt.plot(np.arange(1,len(ypred)+1),yval, marker = 'o', label = 'ytrue')
    plt.plot(np.arange(1,len(ypred)+1),ypred, marker = 'x', label = 'ypred')
    # plt.ylim(min(price), max(price))

    a = np.array(ypred)
    b = np.array(yval)

    mae = np.mean(np.abs(a-b)) 
    plt.title(f'mae: {mae}')

In [ ]:
# MASE implemented courtesy of sktime - https://github.com/alan-turing-institute/sktime/blob/ee7a06843a44f4aaec7582d847e36073a9ab0566/sktime/performance_metrics/forecasting/_functions.py#L16
def mean_absolute_scaled_error(y_true, y_pred):
    """
    Implement MASE (assuming no seasonality of data).
    """
    mae = tf.reduce_mean(tf.abs(y_true - y_pred))

    # Find MAE of naive forecast (no seasonality)
    mae_naive_no_season = tf.reduce_mean(tf.abs(y_true[1:] - y_true[:-1])) # our seasonality is 1 day (hence the shifting of 1 day)

    return mae / mae_naive_no_season

In [ ]:
def result(model, test_data):
    # Make sure float32 (for metric calculations)
    x= test_data.unbatch().as_numpy_iterator()
    y_true = [i[1] for i in list(x)]
    y_true = tf.cast(y_true, dtype=tf.float32)

    y_pred = tf.squeeze(model.predict(test_data))

    y_pred = tf.cast(y_pred, dtype=tf.float32)

    # Calculate various metrics
    mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
    mse = tf.keras.metrics.mean_squared_error(y_true, y_pred) # puts and emphasis on outliers (all errors get squared)
    rmse = tf.sqrt(mse)
    mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
    mase = mean_absolute_scaled_error(y_true, y_pred)

    result =  {"mae": mae.numpy(),
            "mse": mse.numpy(),
            "rmse": rmse.numpy(),
            "mape": mape.numpy(),
            "mase": mase.numpy()}

    print(result)
    return result

## Spliting the dataset ( Train, Test, Val)


In [ ]:
wfull = WindowMaker(price, horizon_size=1, label_columns_names = [wanted])

In [ ]:
windows, horizons = wfull.make_split()

In [ ]:
train_data, val_data = wfull.train_test_val(random_state = 3)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices(train_data).batch(32)
val_data = tf.data.Dataset.from_tensor_slices(val_data).batch(32)

# Model 0: Persistence Algorithm (the “naive” forecast)
The most common baseline method for supervised machine learning is the Zero Rule algorithm.

This algorithm predicts the majority class in the case of classification, or the average outcome in the case of regression. This could be used for time series, but does not respect the serial correlation structure in time series datasets.

The equivalent technique for use with time series dataset is the persistence algorithm.

The persistence algorithm uses the value at the previous time step (t-1) to predict the expected outcome at the next time step (t+1).

This satisfies the three above conditions for a baseline forecast.

To make this concrete, we will look at how to develop a persistence model and use it to establish a baseline performance for a simple univariate time series problem. First, let’s review the Shampoo Sales dataset.



In [ ]:
rand_int = np.random.randint(5,100, (4,1,1,1,4))  
rand_int = np.squeeze(rand_int)

rand_int

array([[94, 96, 35, 80],
       [34, 67, 20, 72],
       [ 7, 45,  7, 53],
       [50, 72, 17, 19]])

In [ ]:
# Baseline
# it just returns a value ahead ( basically predicting the Exact future)

# def model0(label_data):



SyntaxError: ignored

In [ ]:
horizons

In [ ]:
windows.shape, horizons.shape

In [ ]:
val_data

In [ ]:
baseline = Baseline()

baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                 metrics=[tf.keras.metrics.MeanSquaredError(), tf.keras.metrics.MeanAbsoluteError()])

val_performance = {}
performance = {}
# val_performance['Baseline'] = baseline.evaluate(val_data)
# performance['Baseline'] = baseline.evaluate(train_data, verbose=0)

In [ ]:
pred_plot(baseline, val_data, figsize = (25,15))

# Model 1 : a Dense Model

In [ ]:

model1 = keras.Sequential([
    layers.Dense(128, activation = 'relu'),
    layers.Dense(1)
], name = 'model1_dense')

In [ ]:
compile_and_fit(model1, train_data, val_data,epochs = 200, verbose = 0, ckpt_verbose = 0)

In [ ]:
pred_plot(model1, val_data, figsize = (25,15))

The Graph above shows the comparison between actual points and predicted points only. 
Note: Predicted prices are not very diffe

In [ ]:
model1_result = result(model1, val_data)

In [ ]:
x= val_data.unbatch().as_numpy_iterator()
x=list(x)
x[0]

In [ ]:
windows, horizons

In [ ]:
test_size = .2

In [ ]:
test_len = int(len(windows)*test_size)
test_len

In [ ]:
xtrain, xtest, ytrain, ytest = windows[:-test_len], windows[-test_len:], horizons[:-test_len], horizons[-test_len:]

In [ ]:
xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

In [ ]:
windows.shape